#### 一. Why
因为消息队列并不能保证消息能够到达对端, 或是被成功处理. 一次需要一种机制来进行分发和处理确认. 这里所讲的ack和confirm虽然是amqp协议的概念, 但是在其他mq中也有相应实现, ack和confirm的灵感都来自于TCP协议

#### 二. Consumer端
1. 开启手动ack
```scala
// this example assumes an existing channel instance
boolean autoAck = false;
channel.basicConsume(queueName, autoAck, "a-consumer-tag",
     new DefaultConsumer(channel) {
         @Override
         public void handleDelivery(String consumerTag,
                                    Envelope envelope,
                                    AMQP.BasicProperties properties,
                                    byte[] body)
             throws IOException
         {
             long deliveryTag = envelope.getDeliveryTag();
             // positively acknowledge a single delivery, the message will
             // be discarded
             channel.basicAck(deliveryTag, multiple=false);
         }
     });
```

2. 分发(`delivery`)消息的标识`Delivery Tag`  
    * Delivery tags are scoped per channel:    
      区分每次分发是很有必要的, 后续的ack机制也是针对每次分发的确认. 当consumer注册成功后, 消息就会经由`basic.deliver`方法发送出去, 这个方法回携带一个参数`delivery tag`, 它是唯一标识一个channel中的某次`delivery`. 换就话说, `delivery tag`的作用范围是每个`channel`
    * client api中进行ack的方法会使用一个`delivery tag`参数, 表示对哪个delivery做确认  
    * 因为`delivery tag`的作用范围限制在每个`channel`之内, 如果ack发生在其它channel, 则会报错并关闭channel

3. Consumer ack模型   
一个节点在分发消息后, 需要知道这条消息是否被正确处理(至少要知道这条消息是否正确抵达consumer), 而消息的正确处理(或到达)旺旺和很多事情有关: 客户端连接, 客户端程序的处理等等; 所一消息协议通常提供一种方法让consumer能够对某次delivery做通知, 是否开启这个功能需要在consumer订阅的时候进行  

4. 自动ack:  
  主要是考虑到吞吐量, 只要消息被写入到tcp, 就认为该消息已经被正确发送病接受. 这种方式通常被称为`"fire-and-forget"`. 如果consumer的TCP连接在成功分发之前断开了, 则这条消息就丢失了. 因此自动acl是不安全的  


#### 三. Consumer ACK的种类  
1. **Positively Acknowledging Deliveries**   
在进行ack时, Java客户端使用`Channel#basicAck`和`Channel#basicNack`来进行`basic.ack`和`basic.nack`.上面例子中的`channel.basicAck(deliveryTag, multiple=false);`就是正向的ack;   

2. **Acknowledging Multiple Deliveries at Once(累计ACK)**     
  [offical explaination](https://www.rabbitmq.com/confirms.html#consumer-acks-multiple-parameter)

3. **Negative Acknowledgement and Requeuing of Deliveries(重新入队列)**   
有时, 一个COnsumer可能来不及处理收到的消息, 但是其它consumer实例可以, 所以希望将受到的消息重新入队来让其它consumer消费; `basic.reject` 和`basic.nack`就是这样的两个协议
```scala
// negatively acknowledge, the message will be discarded
channel.basicReject(deliveryTag, false);
// requeue the delivery
channel.basicReject(deliveryTag, true);
```

#### 四. Consumer prefetch (qos)  
`channel.basicQos(1)`: qos代表rabbitmq队列中unack的消息的最大条数, 不设置的话rabbitmq会把queue中的所有消息一股脑的发送给consumer, 造成内存溢出; 其次, 配置qos后也会让mq有机会把消息分发给不同的consumer



#### 五. Producer Confirm

[异步producer confirm机制, 利用监听器实现](https://blog.csdn.net/u013256816/article/details/55515234)